## Desarrollo de aplicaciones web con Bottle

### BOOTLE

__1.Introducción__

Bottle es una framework web para Python que no tiene dependencias externas con otras librerías.

Para usarla basta tener el archivo bottle.py en el mismo directorio en el que se crea la aplicación web.


In [ ]:
from bottle import route, run

@route ('/hola')
def hola():
    return "<h1> ¡Hola Mundo!</h1>"

run(host='0.0.0.0', port=8080)

Observar:

* La primera línea importa los módulos route y run de Bottle:

   * El módulo run es usado para ejecutar una aplicación en el servidor de desarrollo.

   * El módulo route indica a la aplicación en que URL hay que solicitar la petición. En las aplicaciones bottle se implementa llamando a una función Python por cada URL solicitada, retornando los resultados de la función al usuario.
   
             from bottle import route, run
   
   * A continuación se define el patrón de la ruta que se va asociar a la función definida posteriormente.

             @route ('/hola')
             
   * La función retorna una cadena HTML que será mostrada en el navegador.

             def hola():
                return "<h1> ¡Hola Mundo!</h1>"
                
   * La línea con la instancia sobre run indica que se ejecute en el servidor de desarrollo. Además el parámetro host=‘0.0.0.0’ indica que el contenido puede ser servido en cualquier ordenador, y el parámetro puerto indica el puerto que se va a usar.
   
             run(host='0.0.0.0', port=8080)

   * Una vez ejecutado el programa se introduce la siguiente url en el navegador:

             http://localhost:8080/hola
             
   * El servidor se puede parar tecleando en el terminal “CTRL-C”



__2.Rutas__

Se pueden asociar más de una ruta a una función:

In [ ]:
from bottle import route, run, template
@route('/')
@route('/hola/<name>')
def greet(name='Pepito'):
    return template('Hola {{name}}, ¿qué tal estás?',name=name)
run(host='0.0.0.0', port=8080)

Se pueden definir rutas dinámicas que encajen con más de una URL a la vez. Para ello se usan términos parametrizados que van entre “&lt;” y “>”. Así en el ejemplo anterior __/hola/&lt;name>__  aceptará diferentes peticiones de acuerdo al valor que tome la variable “name”.

Se pueden usar filtros para definir rutas más específicas. En este caso se usa la sintaxis &lt;nombre:filtro> o bien &lt;nombre:filtro:configuración>. 

Algunos filtros:

:int encaja con digitos y convierte el valor a entero.

:float es similar a :int para números decimales. 

:path encaja todos los caracteres incluyendo el caracter “/” y puede ser usado para encajar más de un segmento de una ruta.

:re permite especificar una expresión regular para el campo configuración. El valor encajado no es modificado . 


__3.Métodos de petición HTTP__

El protocolo HTTP define varios métodos para realizar peticiones: GET, POST, PUT, DELETE or PATCH. Por defecto se usa GET.

Para usarlos se añade la palabra al método route() o bien se utilizan las palabras clave: get(), post(), put(), delete() or patch()

In [ ]:
from bottle import get, post, request #or route

def check_login(user,password):
    if user=="Pepe" and password=="789":
        return True
    else:
        return False

@get('/login') #or @route('/login')
def login():
    return '''
       <form action="/login" method="post">
          Username: <input name="username" type="text" />
          Password: <input name="password" type="password" />
          <input value="Login" type="submit" />
       </form>
    '''

@post('/login') #or @route('/login', method='POST')
def do_login():
    username= request.forms.get('username')
    password= request.forms.get('password')
    if check_login(username,password):
        return "<p> Tu información de login es correcta.</p>"
    else:
        return "<p>Login fallado</p>"

run(host='0.0.0.0', port=8080)

En este ejemplo la URL /login es enlazada a dos funciones diferentes, una para una petición GET y otra para una petición POST. La primera muestra un formulario HTML al usuario y el segundo es invocado cuando se envía un formulario y se chequea la información que el usuario ha entrado en el formulario.

__4.Archivos estáticos__

Los archivos estáticos tales como imágenes no son servidos automáticamente, debiendo añadir una ruta y una función para controlar que los archivos son servidos y dónde se encuentran.

En el ejemplo anterior se sirven los archivos que se encuentran directamente en el directorio /path/to/your/static/files

Para servir archivos en subdirectorios se debe usar un filtro:

Se puede forzar a que un archivo sea descargado o sugerir un nombre al usuario de la siguiente manera:

__5.Páginas de error__

Cuando se produce un error, Bottle muestra información sobre el mismo. Esta información puede ser sobreescrita usando la palabra reservada error() 

El único parámetro que se pasa al gestor de errores es una instancia de HTTPError.

Se puede leer una petición, escribir una respuesta y retornar cualquier tipo de datos excepto instancias HTTPError.

Los gestores de errores solo son usados para aplicaciones que retornan o lanzan excepciones del tipo HTTPError.

__6.Generación de contenidos__

Los tipos de datos que se pueden devolver desde Bottle son los siguientes:

* Diccionarios: Los diccionarios de Python son transformados en cadenas JSON y retornadas al navegador con el Content-Type de la cabecera configurada a application/json.

* Cadena vacía, Falso, None y valores booleanos. Producen una salida vacía con el Content-Length de la cabecera a 0.

* Cadenas Unicode son automaticamente codificadas con el codec especificado en el Content-Type de la cabecera (utf8 por defecto) y es tratada como una cadena de bytes normal.

* Cadenas de Bytes. Bottle retorna una cadena entera y añade un Content-Length  de la cabecera basado en la longitud de la cadena.

* Instancias de HTTPError o HTTPResponse. Tiene el mismo efecto que lanzar una excepción. En caso de un HTTPError, el gestor de errores es aplicado.

* Objetos de archivo. Todas las cosas que tienen un método read() son tratados como un archivo.

__7.Codificación__

Bottle usa el parámetro charset del Content-Type de la cabecera para decidir como codificar las cadenas unicode. Por defecto es text/html; charset=UTF8 y puede ser cambiado usando el atributo response.content_type o configurando el atributo response.charset

Alguna veces Python codifica los nombres de forma diferente a los nombres que son soportados por la especificación HTTP. En estos casos primero se configure la cabecera response.content_type y a continuación se configure el atributo response.charset

__8.Cookies__

Es un trozo de texto almacenado perfil de usuario del navegador. Se puede acceder mediante request.get_cookie() y se puede configurar nuevas cookies con el método request.set_cookie()

El método Request.set_cookie() tiene un conjunto de parámetros:
    
   * max_age: Máxima edad en segundos(Por defecto None)
    
   * expires: Tiempo en que expira(Por defecto None)
    
   * domain: Dominio es es permitido para leer la cookie(Por defecto el dominio actual)
    
   * path: Limita la cookie a un camino dado.
   
   * secure: Limita la cookie a conexiones Https(Por defecto a inactivo)
    
   * httponly: Previene la lectura de la cookie desde javascript.(Por defecto inactivo).

Observaciones:

* Las cookies están limitadas a 4kb de texto en muchos navegadores.

* Muchos usuarios configuran sus navegadores para no aceptar cookies.

* Las cookies son almacenadas en el lado del cliente y no son encriptadas

Las cookies pueden ser firmadas criptográficamente de forma que no pueden ser manipuladas, para lo cual se debe proporcionar una clave de firma via un argumento que sea una clave secreta.

__9.Objeto request__

Las cookies, las cabeceras http , los campos de los formularios HTML y otros datos solicitados se pueden obtener a través del objeto global request.

Bottle usa un tipo especial de diccionario para almacenar los datos de los formularios y las cookies, y sus valores en el diccionario son accesibles como atributos retornando cadenas unicode. Incluso si no tiene valor se devuelve la cadena vacía.

__a.Cookies__

Todas las cookies del clientes son accesible mediante __BaseRequest.cookies__. En el siguiente ejemplo se muestra un simple contador basado en las cookies.

__b.Cabeceras http__

Todas las cabeceras enviadas por el cliente son almacenadas en un diccionario sensitivo a mayúsculas y minúsculas que es accesible desde el atributo __BaseRequest.headers__

__c.Variables Query__

La cadena de consulta como /forum?id=1&page=5, usadas para enviar pares de clave/valor al servidores, se puede acceder a sus valores mediante el atributo __BaseRequest.query__, y mediante __BaseRequest.query_string__ se puede acceder a la cadena entera.

__d.Formularios HTML__

Considerar un formulario típico de html:

El atributo action especifica la URL que recibirá los datos del formulario, el método define se usa GET o POST( si usa GET los valores del formulario serán accesibles a través de la URL y por tanto del método __BaseRequest.query__)

Los campos del formulario transmtidos mediante POST son almacenados en __BaseReques.forms__.

__e.Archivos cargados__

Para subir un fichero al servidor se codifica el formulario de una forma diferente, añadiendo el atributo enctype="multipart/form-data"  a la etiqueta &lt;form>. Además se añade las etiquetas &lt;input type="file" />  para poder seleccionar el fichero.

Bottle almacena los archivos cargados en __BaseRequest.files__ . En el siguiente ejemplo se quiere guardar un archivo al disco:

FileUpload.filename contiene el nombre del archivo sobre el sistema de archivos cliente, y el método FileUpload.save se usa para guardar un archivo en disco. Además se puede acceder al archivo directamente mediante FileUpload.file

__f.JSON__

Algunas aplicaciones envían contenido del tipo application/json al servidor. Este contenido se encuentra en el atributo __BaseRequest.json__

__10.Plantillas__

Bottle dispone de un motor de plantillas. Para usar una plantilla se puede usar la función template(). Se debe proporcionar el nombre de la plantilla y las variables que se desean pasar como argumentos clave/valor

En este ejemplo se carga la plantilla hello_template.tpl y se pasa como argumento la variable name. Las plantillas son almacenadas en memoria , y las modificaciones no tendrán efecto hasta que se limpie la cache.

__11.Modelo MVC__

El modelo MVC establece que en toda aplicación web se debe separar la funcionalidad de la interface de usuario, para lo cual se definen 3 componentes:

* Modelo: Representación de los datos y responsable de almacenar, consultar y modificar los datos.

* Vista: Describe como la información debería ser mostrado al usuario. Es usado para formatear y controlar la presentación de los datos.

* Controlador: Es aquella que realiza el procesamiento decidiendo como responder a las peticiones de usuario.

__Creación de un modelo__

Se va a usar como componente para la persistencia de los datos una base de datos SQLite. Para ello se creará una base de datos con un conjunto de datos.

In [ ]:
import sqlite3
db=sqlite3.connect("libreria.sqlite3")
db.execute("CREATE TABLE Libros (id INTEGER PRIMARY KEY, item CHAR(100) NOT NULL, cantidad INTEGER NOT NULL)")
db.execute("INSERT INTO Libros (item,cantidad) VALUES ('El Quijote', 4)")
db.execute("INSERT INTO Libros (item,cantidad) VALUES ('Drácula', 2)")
db.execute("INSERT INTO Libros (item,cantidad) VALUES ('Guerra y Paz', 30)")
db.execute("INSERT INTO Libros (item,cantidad) VALUES ('Hamlet', 1)")
db.execute("INSERT INTO Libros (item,cantidad) VALUES ('Frankestein', 4)")
db.commit()

In [ ]:
db=sqlite3.connect("libreria.sqlite3")
a=db.execute("SELECT * from Libros")
print(list(a))
db.commit()


__Creación del controlador__

En el controlador se implementa la funcionalidad 
principal de la aplicación:

* Conectarse a la base de datos.

* Obtener los datos de la tabla.

* Llamar a la vista para generar la vista, y mostrar los datos formateados al usuario.


In [ ]:
import sqlite3
from bottle import route, run, template

@route("/libreria")
def mostrar_libros():
    db=sqlite3.connect("libreria.sqlite3")
    c=db.cursor()
    c.execute("SELECT item, cantidad FROM Libros")
    data=c.fetchall()
    c.close()
    output=template("mostrar_libros", rows=data)
    return output

run(host="0.0.0.0", port=8080)

Observar que la línea output = template('mostrar_libros', rows=data) llama a la vista para formatear los datos. La vista es una plantilla denominada “mostrar_libros.tpl” que formatea los datos que le son pasados a través de la variable rows que toma como valor los datos recuperados en la consulta.

__Creación de la vista__

La aplicación buscará una plantilla que coincida con el nombre dado en la función template() y finalizado con la extensión .tpl

El archivo puede mezclar HTML con trozos de código en Python.

Observar:

* Se usa un bucle para crear una tabla que se rellena con los datos del modelo.

* La variable rows que tenía los datos del modelo es accesible desde la plantilla.

* Las líneas en Python van precedidas por “%”, siendo posible acceder a la variables desde HTML usando la sintaxis “{{var}}”

__Ejecución__

Se ejecuta el programa que implementa el controlador, y se visita la url:

            http://localhost:8080/libreria

### EJEMPLOS

__Ejemplo 1__

In [ ]:
from bottle import route, run
@route('/hola')
def hola():
    return "<h1>Hola Mundo</h1>"

run(host="localhost", port=8080, debug=True)

Observar:

* @router permite definir una ruta URL para acceder a la página web. En este ejemplo se accede mediante http://localhost:8080/hola.

* Para cada ruta o conjunto de rutas se define una función que debe devolver el código html que se mostrará al acceder a la ruta especificada.

* La función run() ejecuta un servidor web al que se puede acceder desde localhost en el puerto 8080.


__Ejemplo 2__

In [ ]:
from bottle import route, run

@route('/')
@route('/hola/<nombre>')
def saludo(nombre="Pepito"):
    return "Hola %s, bienvenid@"%nombre

run(host="localhost", port=8080, debug=True)

Una ruta dinámica permite definir rutas utilizando variables, por lo que se puede asociar más de una URL.

En el ejemplo:

* Se han definimos dos rutas que muestran la misma página web.

* La segunda ruta es dinámica ya que el segundo parámetro es una variable.

* El valor de la variable de la ruta se envía a la función. También en este caso se ha utilizado un parámetro con un valor por defecto (“Pepito“) en la función.

__Ejemplo 3__

In [ ]:
from bottle import Bottle,route,run,request
@route('/login') 
def login():
    return '''
        <form action="/login" method="post">
            Username: <input name="usuario" type="text" />
            Password: <input name="password" type="password" />
            <input value="Login" type="submit" />
        </form>'''

@route('/login',method='POST') 
def do_login():
    username = request.forms.get('usuario')
    password = request.forms.get('password')
    if username=="antonio" and password=="antonio":
        return "<p>Login correcto</p>"
    else:
        return "<p>Login incorrecto.</p>"

run(host='localhost', port=8080)

En este ejemplo se definen dos rutas que se corresponden con la misma URL

* La primera se utiliza para enviar información mediante un formulario. 

* La segunda es utilizada cuando se ha enviado información usando el método POST.

* El objeto request almacena la información recibida, de forma que con el método request.forms.get se recupera el valor de loa atributos que aparecen en el elemento input del formulario.

__Ejemplo 4__

In [ ]:
from bottle import Bottle,route,run,request,template
@route('/hola')
@route('/hola/')
@route('/hola/<nombre1>')
def hola(nombre1='Mundo'):
    return template('template_hola.tpl', nombre=nombre1)
run(host='0.0.0.0', port=8080)

Bottle v0.13-dev server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [28/Oct/2019 01:11:53] "GET /hola HTTP/1.1" 200 329
127.0.0.1 - - [28/Oct/2019 01:11:53] "GET /favicon.ico HTTP/1.1" 404 748
127.0.0.1 - - [28/Oct/2019 01:12:02] "GET /hola/antonio HTTP/1.1" 200 365


En este ejemplo la plantilla recibe una variable (nombre), con el nombre que indicado en la URL:

* La variable se usa en el código html mediante  los caracteres {{    }}.
* Con el símbolo % se permite la inclusión de una línea python. Dentro de las plantillas no funciona el tabulado propio de python, por lo hay que indicar el final de los bucles y las condicionales con una instrucción end.

* Se puede hacer uso de los métodos de la clase String ({{nombre.tittle()}})

__Ejemplo 5__

In [ ]:
from bottle import Bottle,route,run,request,template
@route('/suma/<num1>/<num2>')
def suma(num1,num2):
    return template('template_suma.tpl',numero1=num1,numero2=num2)
run(host='0.0.0.0', port=8080)

En este ejemplo se realiza la suma de dos números recibidos en una ruta dinámica con dos variables (num1 y num2), cuyos valores son enviados a la plantilla en dos variables (numero1 y numero2):

* El símbolo % indica una instrucción en Python, y se tiene que usar la instrucción end para indicar el final de la condicional.

* Se usan dos variables: suma donde se guarda la suma de los dos números y resultado dónde se guarda si la suma es positiva o negativa.

* Para escribir variables en html hay que usar los caracteres  {{  y }}

__Ejemplo 6__


In [ ]:
from bottle import Bottle,route,run,request,template
@route('/lista')
def lista():
    lista=["El Quijote","Hamlet","Divina Comedia"]
    return template('template_lista.tpl',lista=lista)
run(host='0.0.0.0', port=8080)

En este ejemplo se envía una lista a través de la variable lista:

* En el ejemplo se recorre la lista mediante un bucle que debe finalizar con la instrucción end para indicar el final del mismo.

* Para escribir el valor de la variable en una línea html se usan los caracteres {{ }}.

* Se puede usar cualquier método de la clase lista desde html.

__Ejemplo 7__

In [ ]:
from bottle import Bottle,route,run,request,template
@route('/dict')
def dict():
    datos={"Nombre":"Maria","Telefono":689933456}
    return template('template_dict.tpl',dict=datos)
run(host='0.0.0.0', port=8080)

En este ejemplo se envía un diccionario a una plantilla mediante la variable dict:

* En el ejemplo se puede ver como se accede a los distintos campos del diccionario como por ejemplo con dict[‘Telefono’].

* Se puede usar cualquier método de los diccionarios para trabajar con ellos.

__Ejemplo 8__


In [ ]:
from bottle import route, default_app, template, run, static_file, error
from lxml import etree
@route('/')
def index():
    doc=etree.parse("madrid.xml")
    muni=doc.findall("municipio")
    return template("index.tpl", mun=muni)
@route('/<cod>/<name>')
def temp(cod,name):
    doc=etree.parse("http://www.aemet.es/xml/municipios/localidad_"+cod+".xml")
    p=doc.find("prediccion/dia")
    max=p.find("temperatura").find("maxima").text
    min=p.find("temperatura").find("minima").text
    return template("temp.tpl",name=name,max=max,min=min)

@route('/static/<filepath:path>')
def server_static(filepath):
    return static_file(filepath, root='static')

@error(404)
def error404(error):
    return 'Error en la página solicitada'

run(host='0.0.0.0', port=8080)

En este ejemplo se muestra una lista de los municipios de la provincia de Madrid y se puede obtener de cada uno de ellos la temperatura máxima y mínima del día actual.

Cuando se accede a la página principal, se muestra una lista de enlaces a  todas las localidades de la provincia de Madrid. Cada enlace tiene la estructura /cod_postal/nombre. Esta información es leída de un archivo xml denominado Madrid.xml y se paso a la plantilla index.tpl

Cuando se accede a la url /cod_postal/nombre, se leen los datos climáticos del municipio de la página del aemet usando el código postal.  La plantilla temp.tpl recibe tras variables: el nombre del municipio, la temperatura máxima y la mínima.

Sobre las plantillas observa que en las plantillas index.tpl como en temp.tpl se usa la función include añadir código html procedente de las plantillas header.tpl y footer.tpl.

La función erro04 permite gestionar el acceso a URL incorrectas.

### HMTL

HTML (HyperText Markup Language) es un lenguaje  diseñado para la publicación de contenidos en Internet. Permite formatear textos, incluir imágenes y animaciones, así como para enlazar con otros contenidos del mismo o de otro servidor.

Características:

* Es un lenguaje interpretado por los navegadores de tal forma que la apariencia de las páginas puede variar entre diferentes navegadores, versiones y plataformas.

* El lenguaje está estructurado mediante etiquetas normalmente emparejadas y encerrando un contenido. Las etiquetas modifican la cualidad o formato del contenido mostrado en la pantalla del navegador.

					<etiqueta>
						contenido	
					</etiqueta>

* Las etiquetas pueden estar anidadas ordenadamente:

            <etiqueta1>
				<etiqueta2>
				   contenido
			      </etiqueta2>
			</etiqueta1>

* Algunas etiquetas sólo están integradas por la etiqueta inicial y no se cierran. Es decir no existe <etiqueta/>

* Las letras de las etiquetas pueden ser mayúsculas o minúsculas indistintamente.

La estructura básica de un documento HTML es:

    <!DOCTYPE html>
	  <HTML>
		<HEAD>
			<TITLE>Título de la página</TITLE>
		</HEAD>
		<BODY>
			Aquí aparecen todas las etiquetas y contenidos
			que se visualizan en el navegador.
		</BODY>
	  </HTML>

Las partes de un documento HTML:

* La primera línea es la declaración del tipo de documento que indica la versión de HTML que se está usando.

* Todo documento HTML empieza con la etiqueta &lt;HTML> indicando el inicio del documento y acaba con la etiqueta &lt;/HTML> para indicar el final del mismo

* El documento en sí está dividido en 2 partes:

    * El encabezado, encerrado por las etiquetas: &lt;HEAD> &lt;/HEAD>.En el encabezado se encuentra toda la información del documento que no se visualiza en el área principal del navegador, como el título de la página que se encuentra entre las etiquetas &lt;TITLE> &lt;/TITLE>.

    * El cuerpo, encerrado por las etiquetas: &lt;BODY> &lt;/BODY>. Dentro del cuerpo se halla todo el contenido que se muestra en el área principal del navegador, como texto, imágenes, etc.

HTML no interpreta los espacios en blanco o los saltos de línea del documento fuente, para lo cual existen caracteres especiales al efecto.

Observar que si se quiere incluir un comentario dentro de la página HTML, se utilizaran los dos símbolos: &lt;!— (para abrir el comentario) y --> (para cerrarlo).

__Instrucciones de formateo__

Instrucción|Significado
--|--
&lt;B> Texto &lt;/B> | Muestra el contenido "Texto" en negrita
&lt;I> Texto &lt;/I> | Muestra el contenido "Texto" en cursiva
&lt;PRE> Texto &lt;/PRE> | Muestra el contenido como si hubiese sido escrito por una máquina de escribir con una fuente de espacio fijo(tipo Courier) respetando espacios y saltos de línea.
&lt;BLOCKQUOTE> Texto &lt;/BLOCKQUOTE> | Destaca una cita dentro del texto general añadiendo márgenes izquierdo y derecho
&lt;P>  | Efectúa un salto de línea y deja una línea en blanco
&lt;BR> | Efectúa un salto de línea únicamente. Este elemento no se cierra
&lt;HR> | Dibuja una línea horizontal. Este elemento no se cierra
&lt;ADDRESS> Texto &lt;/ADDRESS> | Indica el nombre del autor del documento formateando el texto en cursiva y alineando a la izquierda
&lt;H4> Texto cabecera &lt;/H4> | Las etiquetas &lt;Hx> con x desde 1 hasta 6, muestran diferentes tamaños de letras y un salto de línea variable. La cabecera de tipo 1 es la más grande, decreciendo el tamaño de la fuente al aumentar el número x.

__Listas__

Se pueden mostrar listas de contenidos, existiendo 3 tipos:

Estructura| Significados
--|--
&lt;UL> &lt;LI> Palabra 1 &lt;LI> Palabra 2 &lt;LI> Palabra 3 &lt;LI> ....&lt;/UL> | Presenta una lista sin que sus elementos estén precedidos de un número secuencial
&lt;OL> &lt;LI> Palabra 1 &lt;LI> Palabra 2 &lt;LI> Palabra 3 &lt;LI> ....&lt;/OL> | Presenta una lista cuyos elementos están precedidos de un número secuencial
&lt;DL> &lt;DT> Término 1 &lt;DD> Def. término 1 &lt;DT> Término 2 &lt;DD> Def.término 2 &lt;/DL> | Se utiliza para glosarios o definiciones de términos. Cada bloque de texto está formado por 2 etiquetas <DT> que indica la palabra que deseamos definir, y <DD> que es la definición propiamente dicha.

Observar que se pueden anidar unas listas dentro de otras.

__Enlaces__

Permiten conectar contenidos del mismo o de diferentes servidores de manera muy sencilla e intuitiva. Usualmente, los enlaces tienen la siguiente estructura:
		<A HREF=“Destino"> Representación del enlace </A>

Se pueden diferenciar 3 tipos de enlaces:

* Enlaces dentro de una misma página: Son  utilizados para poder saltar dentro de una misma página. La primera línea es la marca en la página (también llamada ancla) e indica el sitio exacto adonde se desea saltar; la segunda línea es el enlace, que nos llevará al ancla.
                <A NAME="marca"> </A>
                <A HREF="#marca"> </A>

* Enlaces a otras páginas: Son  utilizados para poder “enlazar” a páginas o alguna parte  concreta de la página(mediante un ancla) del mismo servidor o a páginas de diferentes servidores. 

                 <A HREF="pagina.htm"> Representación del enlace </A>
                 <A HREF="pagina.htm#marca"> Representación del enlace </A>
                 <A HREF="http://dominio/pag.htm"> Representación del enlace </A>


* Enlaces a una dirección e-mail: Son  usados para facilitar el envío de correo electrónico.
    	<A HREF="mailto:email@dominio">
   			Representación del enlace	
	    </A>

__Imágenes__

Para incluir imágenes se utiliza la etiqueta <IMG> (no tiene etiqueta de cierre)y se indica el nombre del fichero que contiene la imagen a  mostrar. 
                   <IMG SRC="imagen.gif" ALT="descripción“ ALIGN=alineamiento WIDTH=ancho HEIGHT=alto>
                   
Los atributos de la etiqueta &lt;IMG> son:

Atributo | Significado
--|--
SRC | Indica el camino y el nombre de fichero donde se encuentra la imagen que se quiere incluir. Ésta puede residir en el mismo o en otro servidor. Es el único que siempre debe aparecer en la etiqueta.
ALT|Se utiliza para mostrar un texto emergente cada vez que se coloca sobre la imagen el puntero del ratón.
ALIGN|Se emplea para alinear la imagen en la página HTML. Los valores posibles son: TOP, MIDDLE, BOTTOM, RIGHT y LEFT.
WIDTH HEIGHT| Se usan para modificar el tamaño de la imagen en la página HTML. El tamaño se expresa en pixels (por ejemplo, WIDTH = 200) o con un porcentaje del tamaño de la imagen (por ejemplo, WIDTH = 60%). Si no se incluyen estos atributos, la imagen aparece con las mismas dimensiones con las que ha sido creada.

__Fondos__

Se puede cambiar el fondo de una página con un color uniforme o mediante una imagen tápiz. Para ello se introduce un atributo en la etiqueta BODY.

Etiqueta | Significado
--|--
&lt;BODY BGCOLOR="#XXYYZZ" TEXT="#XXYYZZ“ LINK="#XXYYZZ" VLINK="#XXYYZZ" ALINK="#XXYYZZ"> | BGCOLOR: Color de fondo de la página en formato hexadecimal; TEXT : Color del texto; LINK : Color de los enlaces; VLINK: Color de los enlaces visitados; ALINK: Color de los enlaces activos (cuando se pulsan).
&lt;BODY BACKGROUND="Archivo">|BACKGROUND: Permite incluir una imagen en formato gif o jpg, de fondo en la página. "Archivo" hace referencia a la imagen. Además, se pueden añadir todos los modificadores vistos en el modo anterior para establecer los colores del texto y de los enlaces.

__Fuente de un texto__

Para establecer las características de la fuente de un texto, se usa la etiqueta <FONT>:

             <FONT FACE="fuente" SIZE="tamaño"  COLOR="#XXYYZZ">Texto</FONT>

Los atributos de la etiqueta &lt;FONT> son:

Atributo| Significado
--|--
FACE |Se elige el tipo o tipos de fuente para el "Texto" contenido entre la etiqueta de inicio y de cierre.
COLOR |Selecciona el color de la fuente en formato hexadecimal.
SIZE|Se cambia el tamaño de la fuente asignando un valor de 1 a 7 (1 es la más pequeña). El tamaño normal es el 3. Se puede cambiar relativamente respecto a este tamaño normal escribiendo por ejemplo, &lt;FONT SIZE=+1>. También  se puede cambiar el tipo de fuente normal de todo el documento incluyendo a continuación de la etiqueta BODY la etiqueta: &lt;BASEFONT SIZE=5>. 

__Alineación de textos e imágenes__

Cualquier texto o imagen incluidos en una página se puede alinear usando llas etiquetas CENTER, LEFT, RIGHT como por ejemplo:

			             <CENTER>Texto o Imagen</CENTER>
Además en el caso de las imágenes puede usarse el modificador ALIGN de la
etiqueta IMG.

__Tablas__

Se pueden usar tablas para distribuir y presentar contenidos dentro de una página. El esquema básico:

     <TABLE> Inicio de la tabla
	    <TR> Primera fila
		  <TD> Contenido </TD> Primera columna de la primera fila
		  <TD> Contenido </TD> Segunda columna de la primera fila
	    </TR> Final de la primera fila
	    <TR> Segunda fila
		  <TD> Contenido </TD> Primera columna de la segunda fila
		  <TD> Contenido </TD> Segunda columna de la segunda fila
	    </TR> Final de la segunda fila
	  ……
     </TABLE> Final de la tabla
     
Etiqueta|Significado
--|--
&lt;TABLE BORDER=borde WIDTH=largo HEIGHT=ancho BGCOLOR=#XXYYZZ CELLSPACING=sep_celdas CELLPADDING=sep_cont> Filas &lt;/TABLE> |Indican el comienzo y final de una tabla. BORDER establece la anchura del borde de la tabla (BORDER=0 indica que la tabla no tiene bordes. WIDTH, HEIGHT fijan el largo y ancho de la tabla respectivamente. Se expresan en pixels o porcentaje de la dimensión de la pantalla. BGCOLOR establece el color de fondo de la tabla en formato hexadecimal. CELLSPACING fija la separación entre las celdas de la tabla en pixels. Por defecto 2 pixels. CELLPADDING determina la separación entre el borde y el contenido dentro de cada celda en pixels. Por defecto 1 pixel.
&lt;TR>Celdas&lt;/TR>| Señalan el inicio y el final de una fila. Hay que repetirlas tantas veces como filas deseemos obtener.
&lt;TD ALIGN="alineam_horiz" VALIGN="alineam_vert" COLSPAN=extender_horiz ROWSPAN=extender_vert BGCOLOR=#XXYYZZ> Contenido &lt;/TD> | Inicio y el final de una celda con contenido. Hay que repetirlas tantas veces como columnas deseemos obtener. ALIGN establece el alineamiento horizontal del contenido en la celda. Los valores posibles son: CENTER, RIGHT y LEFT. VALIGN establece el alineamiento vertical del contenido en la celda. Los valores posibles son: TOP, BOTTOM y MIDDLE. COLSPAN indica la extensión de la celda en nº de columnas. Así, una celda puede ocupar varias columnas. ROWSPAN indica la extensión de la celda en nº de filas. Así, una celda puede ocupar varias filas. BGCOLOR establece el color de fondo de la celda. Se expresa en formato hexadecimal. Si hemos incluido este atributo en &lt;TABLE>, cambiará el color por defecto de la celda.
&lt;TH> Contenido &lt;/TH>|Establece el inicio y el final de una celda de tipo cabecera. Se distingue de &lt;TD> en que muestra el texto en negrita y centrado. Se pueden aplicar los mismos atributos que en la etiqueta &lt;TD>

Observar que se pueden anidar unas tablas dentro de otras.

__Formularios__

Los formularios permiten al usuario introducir información y enviarla al servidor. Su estructura es la siguiente:

	     <FORM ACTION="acción_URL" METHOD="método“ ENCTYPE="tipo_codificado">
		      Cuerpo del formulario con los diferentes elementos para la introducción de datos
               y los botones de envío o de borrado.	
	     </FORM>

Los atributos de la etiqueta <FORM> son:

Atributo| Significado
--|--
ACTION | Indica la acción que se debe realizar. Normalmente es una URL que apunta a un CGI, Servlet o página dinámica.
METHOD | Método utilizado para el envío de la información. Puede tomar los valores de “GET” (por defecto) o “POST”. Con “GET” el contenido introducido se añade a la URL como si fuera parte de ésta y con “POST” la información va en el cuerpo de datos separadamente.
ENCTYPE | Denota el tipo de codificación utilizada para enviar los datos. Por ejemplo, “TEXT/PLAIN” consigue que las respuestas sean recibidas como un fichero de texto, perfectamente legible y sin codificar.

Los elementos del cuerpo son de 5 tipos:

* Introducción por medio de texto: el usuario debe completar la información que desea enviar al servidor. Tiene la estructura siguiente:

           <INPUT TYPE="xxx" NAME="yyy" VALUE="zzz" SIZE="aaa" MAXLENGTH="bbb">

   Observar que la etiqueta &lt;INPUT> no tiene cierre. 

   Los atributos de la etiqueta &lt;INPUT> son:

Atributo | Significado
--|--
TYPE |Indica el tipo de elemento utilizados para la introducción de información. Los valores posibles son: a) text para establecer el elemento como un ‘edit’ de texto, y b)password es igual que el anterior, pero al escribir sólo se muestran asteriscos.
NAME|Establece el nombre del elemento para poder relacionarlo con su contenido.
VALUE|Fija el contenido inicial del elemento.
SIZE|Establece la longitud del elemento en la pantalla.
MAXLENGTH | Establece el número máximo de caracteres que se pueden introducir.

* Cuando es necesario introducir un texto que pueda alcanzar una gran longitud u ocupar varias líneas, como un comentario, es conveniente utilizar un elemento de texto de múltiples líneas(etiqueta &lt;TEXTAREA>) en vez de &lt;INPUT>:

        <TEXTAREA NAME="yyy" ROWS="número" COLS="número">
             contenido inicial
        </TEXTAREA>
        
   Los atributos de la etiqueta &lt;TEXTAREA> son:

Atributo | Significado
--|--
NAME | Establece el nombre del elemento para poder relacionarlo con su contenido.
ROWS | Representa el número de filas.
COLS | Representa el número de columnas.

* Introducción por medio de menús. El usuario escoja entre varias opciones.La etiqueta &lt;SELECT> se usa para señalar el inicio y el final del elemento menú y las etiquetas <OPTION> delimitan el inicio y final de las diferentes opciones:


		<SELECT NAME="yyy" MULTIPLE SIZE="aaa">
			<OPTION VALUE="bbb" selected> literal_opción 1 </OPTION>
			<OPTION VALUE="ccc"> literal_opción 2 </OPTION>
			<OPTION VALUE="ddd" selected> literal_opción 3</OPTION>
		</SELECT>
    
   Los atributos de la etiqueta &lt;SELECT> son:
   
Atributo | Significado
--|--
NAME | Establece el nombre del elemento para poder relacionarlo con su contenido.
MULTIPLE | Permite que se puedan seleccionar múltiples opciones.
SIZE |Fija el número de opciones visibles en la pantalla sin necesidad de desplazarse. Sólo funciona conjuntamente con MULTIPLE. Si es mayor que 1, aparecerá una lista; en caso contrario, se verá una lista desplegable.

    Los atributos de la etiqueta <OPTION> son:
    
Atributo | Significado
--|-- 
VALUE |Establece el valor que se asigna a la variable.
SELECTED | Si se incluye en alguna de las opciones, esta opción será seleccionada por defecto.

* Introducción por medio de botones: El usuario confirme una opción determinada con Sí o No. Se utiliza un elemento de tipo checkbox con la estructura:

		<INPUT TYPE="checkbox" NAME="yyy" CHECKED>	
        
  donde:

Atributo | Significado
--|-- 
NAME | Establece el nombre del elemento para poder relacionarlo con el contenido.
CHECKED | Especifica el elemento que aparece seleccionado por defecto.

* Si se  quiere que el usuario decida entre varias posibilidades, se utiliza el elemento radiobutton:

		<INPUT TYPE="radio" NAME="yyy" VALUE="zzz" CHECKED>

  donde:

Atributo | Significado
--|-- 
NAME |Nombre del elemento para poder relacionarlo con el contenido. Para asociar varios botones de radio a una única variable se pone a todos el mismo NAME.
VALUE | Especifica el valor que se asignará a la variable.
CHECKED | Fija la única opción seleccionada por defecto.


* Botones de envío y de borrado. Sirven para que el usuario pulse en ellos y envíe la información al servidor o borre los datos introducidos en la página.	

		<INPUT TYPE="tipo" VALUE="zzz">
 
  donde:

Atributo | Significado
--|-- 
TYPE|Establece el tipo de botón. Para el envío de datos escribiremos "SUBMIT" y para borrar los datos de la página "RESET".
VALUE | Especifica el texto que queremos que aparezca en el botón.

* Elemento invisible: Sirven para guardar alguna variable oculta en el formulario para almacenarla en la página activa.
		
        <INPUT TYPE=HIDDEN NAME="yyy" VALUE="zzz">	
        
  donde:

Atributo | Significado
--|--    
NAME | Establece el nombre del elemento para poder relacionarlo con su contenido.
VALUE |  Especifica el valor que se asignará a la variable.

__Marcos__

Un marco es una ventana independiente dentro de la ventana general del navegador. 

Cada marco tiene sus propiedades diferenciadas: bordes y barras de desplazamiento propias. Usando marcos cada página se dividirá, en la práctica, en varias páginas independientes.

Para crear marcos se necesita un documento HTML específico, que llamado documento de definición de marcos donde se especifica el tamaño y la posición de cada marco y el documento HTML que contendrá. Este documento no contendrá la etiqueta &lt;BODY>

La estructura general de un marco es:

	<HTML>
		<HEAD>
			<TITLE>Título de la página</TITLE>
		</HEAD>
		 <FRAMESET ROWS="xxx, yyy" COLS="vvv, zzz"> 
			<FRAME NAME=“marco1" SRC=“pagina1.html">
			<FRAME NAME=“marco2" SRC="página2.html">
			<NOFRAMES>
				MENSAJE
			</NOFRAMES>
		</FRAMESET>
	</HTML>

Donde:

* &lt;FRAMESET>:Se usa para definir la distribución de los marcos en la página principal mediante columnas (COLS) y filas (ROWS). Son anidables, por lo que se pueden incluir unos marcos dentro de otros. Los formatos admitidos:

   * Porcentual: Se indica el porcentaje que ocupará del espacio total disponible.
   
   * Absoluto: Se especifica el tamaño en pixels.

   * Sobre el espacio sobrante: Un asterisco (*) indica que el marco debe ocupar todo el espacio sobrante. Si se pone en varios marcos, se repartirá el espacio equitativamente, y si se quiere que uno tenga más espacio, se debe poner un número delante del asterisco. Así, un marco con un espacio de 3* será tres veces más grande que su compañero con sólo 1*.
   
* &lt;FRAME>: Define las características de un marco, y sus atributos son:

Atributo | Significado
--|--  
NAME|Asigna el nombre al marco para que podamos referirnos a él.
SRC|Indica la URL del documento HTML que ocupará el marco.
SCROLLING|Establece la aparición de barras de desplazamiento en el marco. Su valor por defecto es AUTO (el navegador decide si se ven o no) .Los otros valores son YES y NO.
NORESIZE|Si lo escribimos, el usuario no podrá cambiar el tamaño del marco.
FRAMEBORDER|Establece el borde del marco. Para eliminar el borde su valor debe ser 0.
MARGINWIDTH|Fija los márgenes horizontales dentro de un marco y se expresa en pixels.
MARGINHEIGHT|Fija los márgenes verticales dentro de un marco y se expresa en pixels.

* &lt;NOFRAMES> : Se utilizan para mostrar un mensaje o una página alternativa cuando el navegador del usuario no soporta marcos.

HTML no admite directamente determinados caracteres por compatibilidad internacional, y hay que sustituirlos por códigos:

Código | Resultado
--|--  
&aacute ; , &Aacute ;,&eacute ; , &Eacute ;,etcetera... | á,Á,é,É,í,Í,ó,Ó,ú,Ú
&ñtilde ; | ñ
&iquest ; | &iquest;
&iexcl ; |&iexcl;
&deg ; | &deg;
&ordf ; |&ordf;
&euro ; | &euro;
&copy ; | &copy;
&reg ; | &reg;
&nbsp ; | Espacio en blanco que no puede ser usado pra saltar de línea










   













